In [18]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import re
import requests

In [22]:
nltk.download('stopwords')
nltk.download('punkt')
encoding_type = 'ISO-8859-1'
df = pd.read_csv(r'C:\Users\arceb\PycharmProjects\Projet_DSSS_2023/fullcorpus20210725_DATE.csv', encoding=encoding_type)
df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arceb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arceb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Unnamed: 0,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term
0,1,"Le Figaro, no. 23520",L'agenda cachÃ© de Moscou en Italie,2020-03-30,"Lasserre, Isabelle",LES SYMBOLES comptent parfois plus que les fai...,"Sur les camions militaires qui, en colonnes, ...",Deux semaines pour se ressaisir et s'unir ou p...,Figaro,migrant_migratoire
1,2,Le Monde,RÃ©cit Les trois semaines qui ont chamboulÃ©...,2020-03-27,"Ãric Albert, CÃ©cile Boutelet, JÃ©rÃ´me Gauth...","Londres, Berlin, Rome et Bruxelles corresponda...","- Jeudi 26 mars, les Vingt-Sept devaient se r...","pandÃ©mie touche tout le monde, indiffÃ©remmen...",Monde,migrant_migratoire
2,3,Le Figaro (site web),Ãlargissement de lâUnion europÃ©enne Ã lâ...,2020-03-26,"Tandonnet, Maxime",Fin observateur de la vie politique franÃ§aise...,"contributeur rÃ©gulier du FigaroVox, Maxime Ta...",es yeux et de se prÃ©parer au changement dâu...,Figaro,migrant_migratoire
3,4,Le Monde (site web),CoronavirusÂ : les trois semaines qui ont cham...,2020-03-26,"Eric Albert, JÃ©rÃ´me Gautheret, CÃ©cile Boute...","Jeudi 26Â mars, les Vingt-Sept doivent se rÃ©u...",n visioconfÃ©rence pour la troisiÃ¨me fois en ...,"e la tenue de ses comptes publics, nâest pas...",Monde,migrant_migratoire
4,5,Le Monde (site web),Marc LazarÂ : Â«Â Plus lâItalie fait nation ...,2020-03-25,Marc Lazar,Tribune. LâItalie est le pays qui enregistre...,date du 25Â mars le plus grand nombre de mort...,t de la School of Government de la Luiss (Libr...,Monde,migrant_migratoire
...,...,...,...,...,...,...,...,...,...,...
8021,8035,LibÃ©ration,TUNISIE Ã©vacuation des rÃ©fugiÃ©s,2011-03-04,NaN,Le monde s'organise face Ã la crisedesrÃ©fugi...,"i secoue la Tunisie, que 95 000 personnes ont ...","hÃ©licoptÃ¨res franÃ§ais Mistral, ont Ã©gale...",LibÃ©ration,asylum_refugee
8022,8036,Le Figaro.fr,Libye/UE: 30M d'euros pour les rÃ©fugiÃ©s,2011-03-03,AFP,La Commissaire europÃ©enne Kristalina Georgiev...,"hargÃ©e de la coopÃ©ration internationale, de ...",e Ã Tunis avec le ministre des affaire Ã©tran...,Figaro,asylum_refugee
8023,8037,"La Croix, no. 38872",Courrier,2011-01-18,NaN,rÃ©fugiÃ©s du tchadMerci pour votre article su...,"rÃ©fugiÃ©s dans l'est du Tchad, paru le 27 dÃ...","chaque gÃ©nÃ©ration, on se demande qui, des Fr...",Croix,asylum_refugee
8024,8038,"LibÃ©ration, no. 9175",Reportage L'Ã©trange aller-retour des Karens...,2010-11-11,Arnaud DUBUS,"Ballots sur la tÃªte, les bras chargÃ©s de sac...","de nouilles ou le sac au dos, quelque 15 000 B...",t des Ã©lections du week-end : la junte avait ...,LibÃ©ration,asylum_refugee


In [23]:
df.replace({'Ã§':'ç','Ã©': 'é', 'Ã¨': 'è', 'Ãª': 'ê', 'Ã´': 'ô','Ã': 'à'}, regex=True, inplace=True)
df

,Unnamed: 0,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term
0,1,"Le Figaro, no. 23520",L'agenda caché de Moscou en Italie,2020-03-30,"Lasserre, Isabelle",LES SYMBOLES comptent parfois plus que les fai...,"Sur les camions militaires qui, en colonnes, ...",Deux semaines pour se ressaisir et s'unir ou p...,Figaro,migrant_migratoire
1,2,Le Monde,Récit Les trois semaines qui ont chamboulé l...,2020-03-27,"àric Albert, Cécile Boutelet, Jérôme Gauthere...","Londres, Berlin, Rome et Bruxelles corresponda...","- Jeudi 26 mars, les Vingt-Sept devaient se r...","pandémie touche tout le monde, indifféremment ...",Monde,migrant_migratoire
2,3,Le Figaro (site web),àlargissement de lâUnion européenne à lâ...,2020-03-26,"Tandonnet, Maxime",Fin observateur de la vie politique française ...,"contributeur régulier du FigaroVox, Maxime Tan...",es yeux et de se préparer au changement dâun...,Figaro,migrant_migratoire
3,4,Le Monde (site web),CoronavirusÂ : les trois semaines qui ont cham...,2020-03-26,"Eric Albert, Jérôme Gautheret, Cécile Boutelet...","Jeudi 26Â mars, les Vingt-Sept doivent se réun...",n visioconférence pour la troisième fois en mo...,"e la tenue de ses comptes publics, nâest pas...",Monde,migrant_migratoire
4,5,Le Monde (site web),Marc LazarÂ : Â«Â Plus lâItalie fait nation ...,2020-03-25,Marc Lazar,Tribune. LâItalie est le pays qui enregistre...,date du 25Â mars le plus grand nombre de mort...,t de la School of Government de la Luiss (Libr...,Monde,migrant_migratoire
...,...,...,...,...,...,...,...,...,...,...
8021,8035,Libération,TUNISIE évacuation des réfugiés,2011-03-04,NaN,Le monde s'organise face à la crisedesréfugié...,"i secoue la Tunisie, que 95 000 personnes ont ...","hélicoptères français Mistral, ont également...",Libération,asylum_refugee
8022,8036,Le Figaro.fr,Libye/UE: 30M d'euros pour les réfugiés,2011-03-03,AFP,La Commissaire européenne Kristalina Georgieva...,"hargée de la coopération internationale, de l'...",e à Tunis avec le ministre des affaire étrang...,Figaro,asylum_refugee
8023,8037,"La Croix, no. 38872",Courrier,2011-01-18,NaN,réfugiés du tchadMerci pour votre article sur ...,"réfugiés dans l'est du Tchad, paru le 27 déce...","chaque génération, on se demande qui, des Fran...",Croix,asylum_refugee
8024,8038,"Libération, no. 9175",Reportage L'étrange aller-retour des Karens ...,2010-11-11,Arnaud DUBUS,"Ballots sur la tête, les bras chargés de sache...","de nouilles ou le sac au dos, quelque 15 000 B...",t des élections du week-end : la junte avait d...,Libération,asylum_refugee


In [24]:
df.replace({'Â': '', 'à': 'é', 'â': "'", "à\na": "ç",'à ': 'à'}, regex=True, inplace=True)
df

,Unnamed: 0,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term
0,1,"Le Figaro, no. 23520",L'agenda caché de Moscou en Italie,2020-03-30,"Lasserre, Isabelle",LES SYMBOLES comptent parfois plus que les fai...,"Sur les camions militaires qui, en colonnes, ...",Deux semaines pour se ressaisir et s'unir ou p...,Figaro,migrant_migratoire
1,2,Le Monde,Récit Les trois semaines qui ont chamboulé l...,2020-03-27,"àric Albert, Cécile Boutelet, Jérôme Gauthere...","Londres, Berlin, Rome et Bruxelles corresponda...","- Jeudi 26 mars, les Vingt-Sept devaient se r...","pandémie touche tout le monde, indifféremment ...",Monde,migrant_migratoire
2,3,Le Figaro (site web),àlargissement de l'Union européenne à l'heur...,2020-03-26,"Tandonnet, Maxime",Fin observateur de la vie politique française ...,"contributeur régulier du FigaroVox, Maxime Tan...",es yeux et de se préparer au changement d'un m...,Figaro,migrant_migratoire
3,4,Le Monde (site web),Coronavirus : les trois semaines qui ont chamb...,2020-03-26,"Eric Albert, Jérôme Gautheret, Cécile Boutelet...","Jeudi 26 mars, les Vingt-Sept doivent se réuni...",n visioconférence pour la troisième fois en mo...,"e la tenue de ses comptes publics, n'est pas s...",Monde,migrant_migratoire
4,5,Le Monde (site web),Marc Lazar : « Plus l'Italie fait nation face ...,2020-03-25,Marc Lazar,Tribune. L'Italie est le pays qui enregistre à...,date du 25 mars le plus grand nombre de morts...,t de la School of Government de la Luiss (Libr...,Monde,migrant_migratoire
...,...,...,...,...,...,...,...,...,...,...
8021,8035,Libération,TUNISIE évacuation des réfugiés,2011-03-04,NaN,Le monde s'organise face à la crisedesréfugié...,"i secoue la Tunisie, que 95 000 personnes ont ...","hélicoptères français Mistral, ont également...",Libération,asylum_refugee
8022,8036,Le Figaro.fr,Libye/UE: 30M d'euros pour les réfugiés,2011-03-03,AFP,La Commissaire européenne Kristalina Georgieva...,"hargée de la coopération internationale, de l'...",e à Tunis avec le ministre des affaire étrang...,Figaro,asylum_refugee
8023,8037,"La Croix, no. 38872",Courrier,2011-01-18,NaN,réfugiés du tchadMerci pour votre article sur ...,"réfugiés dans l'est du Tchad, paru le 27 déce...","chaque génération, on se demande qui, des Fran...",Croix,asylum_refugee
8024,8038,"Libération, no. 9175",Reportage L'étrange aller-retour des Karens ...,2010-11-11,Arnaud DUBUS,"Ballots sur la tête, les bras chargés de sache...","de nouilles ou le sac au dos, quelque 15 000 B...",t des élections du week-end : la junte avait d...,Libération,asylum_refugee


### Test 1

In [29]:
import pandas as pd
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import requests

# Set the timeout value in seconds
timeout_value = 1000000000


# Load RoBERTa model and tokenizer
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Tokenization
tokenized_texts = [tokenizer.encode(text, max_length=512, truncation=True) for text in df['Texte']]

# Perform sentiment analysis
results = []
for tokenized_text in tokenized_texts:
    inputs = tokenizer(tokenized_text, return_tensors='pt', padding=True, truncation=True)
    try:
        with requests.Session() as session:
            response = session.post("https://api-inference.huggingface.co/models/",
                                    json=inputs, timeout=timeout_value)
            response.raise_for_status()
            outputs = response.json()
    except requests.exceptions.Timeout:
        print("Request timed out")
        continue
    logits = outputs['logits']
    predicted_class = logits.argmax().item()
    results.append(predicted_class)

# Post-processing
sentiments = ['negative', 'neutral', 'positive']
df['Sentiment'] = [sentiments[result] for result in results]

# Analysis and visualization
sentiment_counts = df['Sentiment'].value_counts()
print(sentiment_counts)
# You can further analyze or visualize sentiment distribution as per your requirement


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/roberta-base/5bde1d28afb363d0103324efeb5afc8b2b397fe5e04beabb9b1ef355255ade81?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1711033328&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMTAzMzMyOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yb2JlcnRhLWJhc2UvNWJkZTFkMjhhZmIzNjNkMDEwMzMyNGVmZWI1YWZjOGIyYjM5N2ZlNWUwNGJlYWJiOWIxZWYzNTUyNTVhZGU4MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=ox8eQqDY4ZmM1DZ-SfnV%7EcEHH9FlNPDbJlukYH5aFFAz4a8qZw9P2MUz2bOhZFrh8aQqBiVSBcagBL-bKlHRSPhEpF1eqwhtNBnBd0d6q8nWmqB-loB7fOeuf%7EHHaOGeUV8CS%7EAIvpkMk0-CofE6Dc6HnVUZO6iKFLiRKi6FicJKCWAqeoKxgl-91-TKhKsxCX9utdy%7ESzmcQLWtjQQfNR7ZmSX0J3n2Xl61MP%7ETaor1edaYffUvLJwWQ9iN-EsiqhYntFPfaJ-phTbJJd0I-BjRGP5lUI-LNWoePW3BzsK9Z6QwmqtKZKURQXkGNILf7OXDeFdju-1GzoDpb79qSQ__&Key-Pair-Id=KVTP0A1DKRTAX: HTTPSConnectio

model.safetensors:  11%|#         | 52.4M/499M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/roberta-base/5bde1d28afb363d0103324efeb5afc8b2b397fe5e04beabb9b1ef355255ade81?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1711033328&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMTAzMzMyOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yb2JlcnRhLWJhc2UvNWJkZTFkMjhhZmIzNjNkMDEwMzMyNGVmZWI1YWZjOGIyYjM5N2ZlNWUwNGJlYWJiOWIxZWYzNTUyNTVhZGU4MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=ox8eQqDY4ZmM1DZ-SfnV%7EcEHH9FlNPDbJlukYH5aFFAz4a8qZw9P2MUz2bOhZFrh8aQqBiVSBcagBL-bKlHRSPhEpF1eqwhtNBnBd0d6q8nWmqB-loB7fOeuf%7EHHaOGeUV8CS%7EAIvpkMk0-CofE6Dc6HnVUZO6iKFLiRKi6FicJKCWAqeoKxgl-91-TKhKsxCX9utdy%7ESzmcQLWtjQQfNR7ZmSX0J3n2Xl61MP%7ETaor1edaYffUvLJwWQ9iN-EsiqhYntFPfaJ-phTbJJd0I-BjRGP5lUI-LNWoePW3BzsK9Z6QwmqtKZKURQXkGNILf7OXDeFdju-1GzoDpb79qSQ__&Key-Pair-Id=KVTP0A1DKRTAX: HTTPSConnectio

model.safetensors:  11%|#         | 52.4M/499M [00:00<?, ?B/s]

KeyboardInterrupt: 

### Test 2

In [32]:
!pip install vaderSentiment

   --------------------------------------- 126.0/126.0 kB 58.8 kB/s eta 0:00:00


In [34]:
import requests
import bs4
# Import BeautifulSoup (to parse what we download)
from bs4 import BeautifulSoup

import pandas as pd


import nltk

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist


def ajoutSentiment(df):


    model = SentimentIntensityAnalyzer()

    def get_sentiment(text):
        scores=model.polarity_scores(text)
        return scores.get('compound')


    for i in range(len(df)):
        positive = get_sentiment(df['Texte'][i])>0.05
        df["Positive"][i]=positive

    df['Positive']=df['Positive'].astype(str)
#Ajout de la colonne l'entreprise qui va nous permettre d'ajouter l'entreprise
#la plus citée, en concluant que c'est l'entreprise sur laquelle l'article est réalisé

In [35]:
ajoutSentiment(df)

KeyError: 'Positive'

### Test 3

In [36]:
!pip install transformers[tf-cpu]

^C


In [37]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

ModuleNotFoundError: No module named 'tensorflow'